### Converting Hiro's data to neuropy format

In [ ]:
from neuropy.core import Neurons, Epoch
import numpy as np
import scipy.io as sio
from pathlib import Path
import pandas as pd

basepath = Path("/data/Clustering/sessions/Hiro/")

sessions = [
    "RoyMaze1",
    "RoyMaze2",
    "RoyMaze3",
    "TedMaze1",
    "TedMaze2",
    "TedMaze3",
    "KevinMaze1",
]
spkdata = sio.loadmat(basepath / "wake_new/wake-spikes.mat")["spikes"]
statesdata = sio.loadmat(basepath / "wake_new/wake-behavior1.mat")["behavior"]
# prdigmdata = basepath/'wake_new/wake-behavior.mat'

for i, name in enumerate(sessions):
    sessdata = spkdata[name][0][0]
    spiketrains = sessdata["time"].squeeze() / 1e6  # in seconds
    neuron_type_id = sessdata["quality"].astype(float).squeeze()
    neuron_type = np.ones(len(neuron_type_id), dtype="U5")
    neuron_type[neuron_type_id < 4] = "pyr"
    # neuron_type[neuron_type_id == 6] = "mua"
    neuron_type[neuron_type_id == 8] = "inter"

    shank_ids = np.array([_.astype(int).squeeze() for _ in sessdata["id"].squeeze()])
    shank_ids = shank_ids[:, 0]

    sessbehav = statesdata[name][0][0]
    paradigm_epoch = (sessbehav["time"][0][0]) / 1e6  # in seconds
    states_epoch = sessbehav["list"][0][0]
    states_epoch[:, :2] = states_epoch[:, :2] / 1e6  # in seconds
    states_type = np.ones(states_epoch.shape[0], dtype="U4")
    states_type[states_epoch[:, 2] == 1] = "NREM"
    states_type[states_epoch[:, 2] == 2] = "REM"
    states_type[states_epoch[:, 2] == 3] = "QW"
    states_type[states_epoch[:, 2] == 4] = "AW"

    neurons = Neurons(
        spiketrains=spiketrains,
        t_start=paradigm_epoch[0, 0],
        t_stop=paradigm_epoch[-1, -1],
        shank_ids=shank_ids,
        neuron_type=neuron_type,
    )

    if paradigm_epoch.shape[0] > 3:
        paradigm_epoch[1, 1] = paradigm_epoch[2, 1]
        paradigm_epoch = np.delete(paradigm_epoch, 2, 0)

    paradigm = Epoch.from_array(
        paradigm_epoch[:, 0], paradigm_epoch[:, 1], ["pre", "maze", "post"]
    )

    brainstates = Epoch.from_array(states_epoch[:, 0], states_epoch[:, 1], states_type)

    sesspath = basepath / name
    # sesspath.mkdir()
    neurons.save(sesspath / f"{name}.neurons.npy")
    paradigm.save(sesspath / f"{name}.paradigm.npy")
    brainstates.save(sesspath / f"{name}.brainstates.npy")
